In [1]:
from pathlib import Path
import pdfmb

In [2]:
pdfmb.merge(
    pdfs_to_merge=Path("example pdfs").rglob("*.pdf"),  # filter of combine different sources
    output_folder=Path("output"),
)


In [3]:
pdfmb.add(
    pdfs_to_add=Path("example pdfs").rglob("*.pdf"),  # filter of combine different sources
    existing_pdf=Path("example pdfs/root1.pdf"),
)


In [4]:
pdfmb.merge_from_folder(
    source_folder=Path("example pdfs"),
    output_folder=Path("output"),
    add_flat_hierachy=True,
)


In [5]:
pdfmb.add_from_folder(
    source_folder=Path("example pdfs"),
    existing_pdf=Path("example pdfs/root1.pdf"),
    add_flat_hierachy=True,
)


In [4]:
from pathlib import Path
from pikepdf import Pdf, OutlineItem

source_folder = Path("example pdfs")

dirs = {}

pdfs = sorted(source_folder.rglob("*.pdf"))

for pdf in pdfs:
    for d in pdf.parents:
        if d not in dirs and str(d) != str("."):
            dirs[d] = OutlineItem(d.name, 0)

p = Pdf.new()
p.add_blank_page()
page_count = len(p.pages)

with p.open_outline() as outline:
    outline.root.append(dirs[source_folder])

    for file in pdfs:
        src = Pdf.open(file)
        pages = len(src.pages)

        ol_file = OutlineItem(file.name, page_count)

        dirs[file.parent].children.append(ol_file)

        if pages > 1:
            for page in range(pages):
                ol_file.children.append(
                    OutlineItem(f"Page {page+1}", page_count + page)
                )

        page_count += pages
        p.pages.extend(src.pages)

    for d in dirs:
        if d.parent in dirs:
            dirs[d.parent].children.append(dirs[d])


p.save("filename.pdf")
p.close

<bound method Pdf.close of <pikepdf.Pdf description='empty PDF'>>